https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.boxcox.html

При решении задач регрессии используются именно регрессоры. Из хороших классических алгоритмов - SVM

В этой тетради мы будем исследовать наш df  на предмет пропусков, нулевых значений выбросов и занчений ,которые противоречат физике

In [ ]:
# Импортируем в проект требуемые библиотеки

import pandas as pd # библиотека для обработки и анализа данных
import numpy as np
import seaborn as sns # библиотека для создания статистических графиков
import random #  генераторатор случайных чисел и данных
import os # библиотека функций для работы с операционной системой.


import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
%matplotlib inline
sns.set_style('darkgrid')


import ydata_profiling # pandas_profiling!!!! отменяется c 1-го апреля 2023 г

import warnings
warnings.filterwarnings("ignore")
matplotlib.style.use('seaborn-pastel')


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import scipy

from scipy import stats # для метода обнаружения Q и IQR,  а также z-корреляции
from scipy.stats import mstats

np.random.seed(45) # чтобы выборки данных всегда были одни и теже берем любую цифру . мне нравится 45!


# Этап 3.3.

# Проверка того, есть ли в структуре данных какие-либо пропущенные значения

# Прочитаем наш df - 'data_composite_inspect.csv' из тетради Diploma1_data

In [ ]:
data_composite_inspect = pd.read_csv(r'C:\Users\grain\Work_folder\Diplom_MGTU\Diploma1_data\data_composite_inspect.csv')
data_composite_inspect.head()

In [ ]:
data_composite_inspect.describe()

In [ ]:
data_composite_inspect.info()

В нашем файле 1023 строки и 13 признаков-столбцов. Все значения у нас числовые, тип данных float

In [ ]:
Проверим отсутствие: пропущенных и нулевых значений, а также дубликатов строк

In [ ]:
# метод .isnull() выдает логический массив, где пропуски обозначены как True.
# Функция .isnull() используется для проверки того, есть ли в структуре данных какие-либо пропущенные значения.
# метод .sum() по умолчанию суммирует эти True или единицы по столбцам (axis = 0)
# Можно использовать функцию .isnull() вместе с .sum(), чтобы увидеть количество пропущенных значений в каждом столбце.

data_composite_inspect.isnull().sum()

Пропущенных значений нет!

Проверка того, есть ли в структуре данных какие-либо отсутствующих значений (NaN)

In [ ]:
# Функция .isna() в Pandas используется для обнаружения отсутствующих значений (NaN), 
# значения NaN в структуре данных сопоставляются с True, а значения, отличные от NaN, сопоставляются с False.
# Можно использовать функцию isna вместе с sum, чтобы увидеть количество пропущенных значений в каждом столбце.

data_composite_inspect.isna().sum() 

In [ ]:
# Еще один метод определения пропущенных значений - Тепловая карта пропущенных значений sns.heatmap

cols = data_composite_inspect.columns[:13] # первые 13 колонок
# определяем цвета 
# желтый - пропущенные данные, синий - не пропущенные
colours = ['#000099', '#ffff00'] 
sns.heatmap(data_composite_inspect[cols].isnull(), 
            cmap=sns.color_palette(colours)
           )

Мы определили, что в нашем DataFrame data_composite_inspect нет  пропущенных значений и отсутствующих значений (NaN)

Проверим наш DataFrame на наличие дубликатов  строк 

In [ ]:
# Найдем ,если они есть количество дублируеых строк
count_dupl = len(data_composite_inspect)-len(data_composite_inspect.drop_duplicates())

In [ ]:
print(count_dupl)

В нашем dataFrame НЕТ дубликатов

# Продолжаем исследование нашего df - data_composite_inspect

In [ ]:
# применим метод .describe() к количественным признакам
# округлим значения до 2-х знаков после запятой - round(3)
data_composite_inspect.describe(percentiles = [0.25, 0.5, 0.75], include='all').round(2)

In [ ]:
# Полный анализ нашего df  можно осуществить с помощью модуля df.profile_report()
data_composite_inspect.profile_report()

#Графический анализ

In [ ]:
# Взято из https://pyprog.pro/sns/sns_6_visualization_of_dist.html
# Создаем экземпляр класса:
g = sns.PairGrid(data_composite_inspect) #.head(100)) # head(100) можно ли вставить  этот параметр???

# задаем тип графиков над
# главной диагональю:
g.map_upper(sns.scatterplot, alpha=0.5) #, hue = 'pattern_angle')

# задаем тип графиков под
# главной диагональю:
g.map_lower(sns.kdeplot, bw_adjust=0.7) #, hue = 'pattern_angle')

# задаем тип графиков на
# главной диагонали:
g.map_diag(sns.histplot, kde=True, bins = 30);

In [ ]:
# Построение графиков
# Взято из https://pyprog.pro/sns/sns_6_visualization_of_dist.html

# Функция построения графиков по парметрам df:  sns.histplot и sns.boxplot
# Параметры, передаваемые в функцию:
# df- наш dataframe
# plt_type - тип графика ,который выводится (hist - histplot, box - boxplot, dis - displot. kde - kdeplot)
# n_row - количестворядов графиков
# n_col - количество столбцов графиков

def drawing_graphs(df, plt_type, n_row, n_col):
    
    fig, axes = plt.subplots(n_row, n_col) # в нашем примере : 2, 2
    i = 0              # количество графиков во фрейме
    n_colum  = df.shape[1]  # количество столбцов в df / если 1, то количество столбцов (13), если 0, то строк 1023 !
    
    for column in df.columns:
        if plt_type = hist:
            sns.histplot(df[column],ax=axes[0, i],
                         bins = 30, 
                         kde = True, # оценкf плотности ядра (kernel density estimation или сокращенно kde)
                         fill = True
                         )    
        elif  plt_type = box:       
            sns.boxplot(data=df, x=column, ax=axes[1, i])
        elif plt_type = kde:
             sns.kdeplot(df[column], ax=axes[0, i], 
                         bins = 30,
                         bw_adjust=0.7
                        ) 
        elif plt_type = dis:
             sns.displot(data =df, x = column, 
                         kind = 'kde',
                         fill = True
                        )
        elif plt_type = scatter
            sns.scatterplot(df[column], ax=axes[0, i],
                            alpha=0.5
                           )
        i = i + 1
        if i > (n_col - 1) or column == df.columns[n_colum - 1]:
            plt.show()
            fig, axes = plt.subplots(n_row, n_col)
            i = 0
    return

In [ ]:
drawing_graphs(data_composite_inspect, 2, 2)

Нужно ли?
i = 1
for column1 in df.columns:
    for column2 in df.columns[i:]:
        sns.jointplot(data=df, x=df[column1], y=df[column2], hue="Угол нашивки, град")
        plt.show()
    i = i + 1

In [ ]:
# История с  логорифмом __________________________________________

Взято https://habr.com/ru/post/557998/
Данные гистограммы параметра 'surface_density' показывают четко выраженную асимметрию. 
Хвост с правой стороны намного длиннее, чем с левой, и поэтому мы говорим, что асимметрия - положительная. 
Мы можем оценить асимметрию данных количественно при помощи функции библиотеки pandas skew:

In [ ]:
data_composite_inspect['surface_density'].skew()

In [ ]:
# Эта асимметрия может быть эффективным образом смягчена 
# путем взятия логарифма веса при помощи функции библиотеки numpy np.log

In [ ]:
# А теперь уберем ассиметрию в 'surface_density'с помощью (np.log) предварительно создав новый df: df_log

In [ ]:
df_log = data_composite_inspect.copy()

In [ ]:
df_log['surface_density'] = np.log(df_log['surface_density'])

In [ ]:
# Выведем первоначальную гистаграмму 'surface_density' в df: data_composite_inspect

In [ ]:
data_composite_inspect['surface_density'].hist(bins=30)

In [ ]:
# а теперь гистограмму 'surface_density' из df: df_log
df_log['surface_density'].hist(bins=30)

In [ ]:
df_log['surface_density'].skew()

In [ ]:
# гистограмма 'surface_density' сместилась вправо и стала отризательной ,но мы ушли от значений 0!

In [ ]:
'''Визуализация корреляции '''
 
xs = data_composite_inspect['strapery_strength']
ys = data_composite_inspect['surface_density'].apply( np.log )
pd.DataFrame(np.array([xs,ys]).T).plot.scatter(0, 1, s=12, grid=True)
plt.xlabel('strapery_strength')
plt.ylabel('Логарифмическая surface_density')
plt.show()

In [ ]:
# Конец истории с логарифмом__________________________________

In [ ]:
# Графики зависимости параметров друг от друга с плотностью распределения
# Нужно ли?
sns.pairplot(data_composite_inspect.head(100), diag_kind='kde') #, hue='pattern_angle')

In [ ]:
# Проведем анализ данных в нашем df: data_composite_inspect на предмет 'подозрительных значений' и выбросов

При анализе df.discribe  и df.profile_report() выяснилось ,что в столбцах 'Шаг нашивки' и 'Плотность нашивки'
min значения равны 0.0 ,что не может быть физически.

А также значения MIN в столбцах:'ratio_filler_matrix'  = 0.389 'этот параметр  не может быть меньше 1,
'elasticity_module'= 2.437 это значение на 2 порядка меньше среднего- явная ошибка заполнения,
surface_density = 0.604 то значение на 3 порядка меньше среднего- явная ошибка заполнения.

Эти МИН значения несоотвествуют физически
м параметрм нашего проекта относительно показателей для композитных материлов.
И являютя выбросми 'слева'.

In [ ]:
def column_value_counts(df):
    for column in df.columns:
        print('столбец : ', column)
        print('Интервал значений', '  ', 'Количество в интервале')
        print(df[column].value_counts(bins=30))
        print('-----------------------------')
    return

In [ ]:
column_value_counts(data_composite_inspect)

In [ ]:
# Мы можем определить интервалы значений, которые можно отбросить  в первом приближении по МИН значению,
# как противоречищий физическому смыслу: 
# 1-й интервал (0.527, 66.074]    строк 16
# 2-й интервал :
'''
1529.716, 1593.353]     5
(1593.353, 1656.989]     3
(1784.263, 1847.899]     1
(1847.899, 1911.536]     1
(1656.989, 1720.626]     0
(1720.626, 1784.263]     0
'''
#  и так получим значения крайних -левых значений для DataFrame для каждого столбца в df_min_max!!!

In [ ]:
# Создадим DataFrame  df_min_max который содержит мин и мах значения по каждому столбу в качестве критерия отбрасывания строк 
# min - индекс сроки 0 и max- индекс строки 1 получены оценочным путем при исследовании гистограмм по каждому параметру-столбцу
# Для столбца 'pattern_angle'диапазон будет :[0.0, 90.0],

df_min_max = pd.DataFrame({
                   'pattern_angle':[0.0, 90.0],  
                      'step_strip':[1.5, 13.5],
                  'density_strip' :[29.0, 85.6],
             'ratio_filler_matrix':[0.5, 6.0],
                        'density' :[1780.0, 2159.0],
              'elasticity_module' :[50.0, 1560.0],
                'number_hardeners':[30.0, 190.0],     
            'content_epoxy_groups':[16.0, 28.0],
               'flash_temperature':[180.0, 390.0],
                'surface_density' :[20.0, 1200.0],
    'elasticity_module_stretching':[66.0, 81.0],
              'strapery_strength' :[1326.0, 3600.0],
               'resin_consumption':[80.0, 350.0]          
               })

df_min_max

In [ ]:
# Функция расчета СРЕДНЕГО значения для "подозрительного элемента" в столбце с разбросом 
# как ВЫШЕ так и НИЖЕ от "подозрительного элемента"
# Передаваемые в функцию параметры:
# df[column] - столбец значений в котором сейчас идет проверка
# n_row -      номер строки "подозрительного элемента"
# n_spread -   задаваемый разброс обычно я задаю 5.

def mean_round_point(df, column, n_row, n_spread):
    mean_n_spread = 0 # возвращаемое значение среднего, полученного из суммы ближайших (n_spread * 2) значений столбца
    
    higth_value_sum = 0 # Сумма значений, где текущее значение номера строки Меньше (n_spread) 
    low_value_sum = 0   # Сумма значений,  где текущее значение номера строки Больше (n_spread)
    
# Для значений столбца, где текущее значение номера строки Меньше (n_spread) - заданного интервала для определения СРЕДНЕГО значения
    for n in range(n_spread):
        if (n_row == 0) or (n_row - n < 0):
            break
        else:
            higth_value_sum = higth_value_sum + df.loc[n_row - n][column]
            n = n + 1    
# Для значений столбца, где текущее значение номера строки Больше (n_spread) - заданного интервала для определения СРЕДНЕГО значения
    for k in range(n_spread):   
        if n_row + k >= len(df[column]):  
            break
        else:
            low_value_sum = low_value_sum + df.loc[n_row + k][column]    
            k = k + 1
# Рассчитываем суммы n_spread элементов выше (left_value_sum)  и ниже (rigth_value_sum) 'подозрительного элемента' строки n_row
    mean_n_spread =  (higth_value_sum + low_value_sum) / (n + k)
    print('новое значение :', mean_n_spread)
    return mean_n_spread

In [ ]:
def clean_df_left(df, df_min_max):
    sum_n_quantity = 0 # Общее количество подозрительных строк в df

    for column in df.columns:
        n_col = column
        print('подозрительный столбец : ', n_col)
        print('экспертное минимально-допустимое значение :', df_min_max.loc[0, n_col])
    
        n_quantity = 0 # Количество подозрительных строк в столбце
       
        for i in range(len(df)):
            if df.loc[i, n_col] < df_min_max.loc[0, n_col]:
                n_quantity =  n_quantity + 1
                print('строка N:', i)
                print('старое значение :', df.loc[i, n_col])      
            # Функция 'mean_round_point' расчета СРЕДНЕГО значения для "подозрительного элемента" в столбце с разбросом '5'
                df.at[i, n_col] = mean_round_point(df, n_col, i, 5) 
        print('Количество подозрительных строк столбце = ', n_quantity)
        print('---------------------------------')
        sum_n_quantity = sum_n_quantity + n_quantity
    print('Общее количество подозрительных строк в df = ', sum_n_quantity)
    return(df)

In [ ]:
clean_df_left(data_composite_inspect, df_min_max)

всего замена подозрительных значений слева произведена в 106 строках

Проведем очистку нашег df (в котором 1023 строки) от выбросов

Способ удаления выбросов IQR.
Записшем очищеный массив в df - data_clean

In [ ]:
# https://www.codecamp.ru/blog/remove-outliers-python/
# find Q1, Q3, and interquartile range for each column
# создаем новый df: data_clean

def outliers_delit(df):
    Q1 = df.quantile(q=.25)
    Q3 = df.quantile(q=.75) 
    IQR = df.apply(stats.iqr) # from scipy import stats

#only keep rows in dataframe that have values within 1.5\*IQR of Q1 and Q3

    data_clean = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

#find how many rows are left in the dataframe 

    data_clean.shape
    print('Количество строк до удаления: ', len(df))
    print('Количество строк после удаления: ', len(data_clean))
    print('Удалено строк :', (len(df) - len(data_clean)))
    
    return(data_clean)

In [ ]:
data_clean = outliers_delit(data_composite_inspect)

#### У нас было 1023 строки  до применения метода IQR. Теперь мы получили после очистки 936 строк в df -  data_clean.
Удлили 61 строк

второй способ z-с  посмотрим сколько строк уберет этот метод из  массива data_clean

In [ ]:
## Defining the function to calculate the Z - Score
def Z_score(data):
    global outliers,zscore
    outliers = []
    zscore = []
    upper_threshold = 3
    lower_threshold = -3
    mean = np.mean(data)
    std = np.std(data)
    for i in data:
        z_score= (i - mean)/std 
        zscore.append(z_score)
        if np.abs(z_score) > upper_threshold or np.abs(z_score) < lower_threshold:
            outliers.append(i)
    return print("Total number of outliers are",len(outliers)) #outliers,,  zscore

In [ ]:
## Function calling
for column in data_clean.columns:
    Z_score(data_clean[column])
    

Данный метод Z - Score НЕ нашел возможных для удаления строк!

Проведем  графический анализ имеющихся данных в data_clean методом sns.boxplot

In [ ]:
# Функция построения графиков по парметрам df:  sns.boxplot
# Параметры, передаваемые в функцию:
# df- наш dataframe
# n_col - количество столбцов графиков

def drawing_graphs_boxplot(df, n_col):
    m = df.shape[1] 
    i = 0
    fig, axes = plt.subplots(1, n_col, figsize=(12, 2))
    for column in df.columns:
        sns.boxplot(data=df, x=column, ax=axes[i])
        i = i + 1
        if i > (n_col - 1) or column == df.columns[m - 1]:
            plt.show()
            fig, axes = plt.subplots(1, n_col, figsize=(12, 2))
            i = 0
    return                      

In [ ]:
drawing_graphs_boxplot(data_clean, 3)

У нас в 2-х столбцах наблюаются выбросы после замены 'подозрительных значений' СПРАВА на средние значения +/- 5 значений в столбце и очистки стандартными методами IQR и Z-C.
Проведем дополнительную очистку  в столбцах: 'elasticity_module_stretching' и 'strapery_strength'.
Используем функцию "Методом подбора коэффициентов в df_min_max" и удалением этих выбросов в df.

Проведем иттерацию по удалению строк из нашего df со значениями в df_min_max  при max- строка '1'.

Проведем очистку каждого парметра от значений  'max' лежащих дальше чем 3 СИГМА  'elasticity_module_stretching' и 'strapery_strength'.

In [ ]:
# Функция очистки значений в стлбцах СПРАВА
# пердаваемые параметры:
# df - наш массив
# df_min_max - массив экпертных крайних сначений для каждого столбца
# df_name_column - массив наименований столбцов ,в которых необходимо провести очистку СПРАВА

def clean_column_rigth(df, df_min_max, df_name_column):
    sum_del_rigth = 0 # Количество строк удаленных выбросов СПРАВА после определения столбцов с выбросами в boxplot
    # df_name_column = pd.DataFrame(columns=list_name_column)
    for name_col in df_name_column.columns:
        for column in df.columns:
            if column != name_col:
                continue
            l_begin = len(df)
            df = df[(df[column] < df_min_max.loc[1, column])]     
            l_end = len(df)
            list_x = l_begin - l_end
            print(df.shape)
            print('Количество подозрительных строк в столбце = ', list_x)
            print('---------------------------------')
        sum_del_rigth = sum_del_rigth + list_x
    print('Общее количество удалнных подозрительных строк из-за выбросов справа в df = ', sum_del_rigth)
    #data_clean = data_clean_def  
    return(df)

In [ ]:
# Создадим список столбцов и df, где есть выбросы СПРАВА для передачи в  функцию clean_column_rigth
list_name_column = ['elasticity_module_stretching', 'strapery_strength']
df_name_column = pd.DataFrame(columns=list_name_column)

In [ ]:
data_clean = clean_column_rigth(data_clean, df_min_max, df_name_column)
data_clean.shape

Удалили 11 строк с выбросами справа.

In [ ]:
print('Количество строк до очистки df: ', len(data_composite_inspect))
print('Количество строк после очистки: ', len(data_clean))
print('Удалено строк :', (len(data_composite_inspect) - len(data_clean)))

В начальном DF  у нас было 1023 строки, сейчас осталось 951 строк. Всего удалили  72 строк на данной иттерации

In [ ]:
# Опять проверим на выбросы наш df  на примере графиков 'ящиков с усами' для каждого столбца

In [ ]:
drawing_graphs_boxplot(data_clean, 3)

В нашем df опять появились выбросы после удаления 11 из-за выбросов СПРАВА по функции clean_column_rigth

In [ ]:
# DataFrame 'df_min_max' который содержит мин и мах значения по каждому столбу в качестве критерия отбрасывания строк 
# min - индекс сроки 0 и max- индекс строки 1 получены оценочным путем при исследовании гистограмм по каждому параметру-столбцу
# Изменим значение в df_min_max в столбце  'elasticity_module' :[50.0, 1560.0] на значения 'elasticity_module' :[50.0, 1550.0]

df_min_max =  df_min_max.replace({'elasticity_module' : { 50.0 : 50.0, 1560.0 : 1550.0}})
df_min_max

In [ ]:
# Следующая иттерация по удалению выбросов СПРАВА 
# Список столбцов, где есть выбросы СПРАВА для передачи в  функцию clean_column_rigth
list_name_column = ['elasticity_module']
df_name_column = pd.DataFrame(columns=list_name_column)

In [ ]:
# Повторно вызываем функцию clean_column_rigth
data_clean = clean_column_rigth(data_clean, df_min_max, df_name_column)
data_clean.shape

В столбце 'elasticity_module' удалили еще 3 строки

In [ ]:
# Проведем еще одну иттерацию проврки df с  помощью boxplot

In [ ]:
drawing_graphs_boxplot(data_clean, 3)

# У нас получились "чистые ", без выброов "ящики с усами"!
Перейдем к следующему этапу исследования нашего datafrema

In [ ]:
print('Количество строк до очистки df: ', len(data_composite_inspect))
print('Количество строк после очистки: ', len(data_clean))
print('Удалено строк :', (len(data_composite_inspect) - len(data_clean)))
total_del_percent = ((len(data_composite_inspect) - len(data_clean)) / len(data_composite_inspect)) * 100
print('Удалено : ', total_del_percent, '%')

В начальном DF  у нас было 1023 строки, сейчас осталось 948 строк. Всего удалили  75 строк или 7.33 %.

# Запишим data_clean в новый файл 'data_main.csv' для дальнейшей работы и передачи в pipline и изучением методами sklearn

In [ ]:
# Запишим data_clean в новый файл 'data_main.csv' для дальнейшей работы и передачи в pipline и изучением методами sklearn
# В массиве 'data_main.csv' мы:
# 1. Заменили нулевые значения '0.0' в строке 19 столбцов 'Шаг нашивки' - 'step_strip'
# и 'Плотность нашивки' - 'density_strip'
# 2.Удалили параметр 'Угол нашивки' - 'pattern_angle' как не несущий существенной информации , так как там всего 2 значения угла  и 90 град распределенных по 50% 
# 3. Провели очистку выбросов методом IQR  и удалили еще 76 строк.
# 4. Метод  Z - Score не нашел дополнительные выбросы
# 5. Проведя графический аналих по boxpot,  мы убедились что в ряде столбцов остались выбросы. Чтобы убрать оставшиеся выбросы,
#  мы в ручном режиме в цикле убрали по шаблону df_min_max значния меньше МИН и больше МАХ . Всего еще 21 строку.
# Всего мы удалили 1023 - 941 = 82 строк  или 8 % из исходного файла 
# Считаем, что df очищен от выбросов

data_clean.to_csv('data_main.csv', index=False, float_format="%.3f")

#  Этап 4 Нормализуем и стандартизируем наш dataset, чтобы привести наши данные к близким размерностям

# В нашем DataSet "data_composite_inspect" есть 12 признаков- столбцов
Но все признаки имеют разную размерность.
Нормализуем и стандартизируем  наш массив 


# Этап 4.1 Нормализация по методу MinMaxskaler

In [ ]:
# список наших столбцов
list_data = list(data_clean)

In [ ]:
minmax_scaler = MinMaxScaler() # вызываем метод MinMaxskaler

In [ ]:
#  обучаем MinMaxScaler() -передаем только числовые значения столбцов  ,указанных списком list_data или можно перечислить какие нужны
data_clean_norm = minmax_scaler.fit_transform(np.array(data_clean[list_data])) 
                                                           

In [ ]:
data_clean_norm[:1] # Проверим первую строку

In [ ]:
# создадим новый df с нормализованными данными с названием наших столбцов 
data_clean_norm_df = pd.DataFrame(data = data_clean_norm, columns = list_data)
data_clean_norm_df.head()

In [ ]:
data_clean_norm_df.describe()

In [ ]:
data_clean_norm_df.to_csv('data_main_norm.csv', index=False, float_format="%.3f")

# 4.2 Проведем стандартизацию по методу StandardSckaler

# from sklearn.preprocessing import StandardScaler

In [ ]:
std_skaler = StandardScaler()

In [ ]:
#  обучаем StandartSkaler() - передаем только числовые значения столбцов  ,указанных списком list_data или можно перечислить какие нужны
data_clean_std = std_skaler.fit_transform(np.array(data_clean[list_data])) 
                                                           

In [ ]:
data_clean_std[:1]

In [ ]:
# создадим новый df со стандартизованными данными с названием наших столбцов  (вернем их)
data_clean_std_df = pd.DataFrame(data = data_clean_std, columns = list_data)
data_clean_std_df.head()

In [ ]:
data_clean_std_df.describe()

In [ ]:
data_clean_std_df.to_csv('data_main_std.csv', index=False, float_format="%.3f")

sns.displot(data =data_composite_inspect) #, kind = 'kde', fill = True) # = bin_s # hue="cut"
plt.show()

In [ ]:
# примеры
